In [ ]:
!pip install gtfs-realtime-bindings geopandas contextily

In [2]:
from google.transit import gtfs_realtime_pb2
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import contextily as cx
import matplotlib.pyplot as plt
from glob import glob
from tqdm.notebook import tqdm

In [4]:
filename = "./data/1-koda-gtfs-rt-extracted/"

response = open(filename, 'rb')
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.read())

print(len(feed.entity))

print(feed.entity[2])

FileNotFoundError: [Errno 2] No such file or directory: './data/1-koda-gtfs-rt-extracted/2021-04-01/sl/VehiclePositions/2021/04/01/18/sl-vehiclepositions-2021-04-01T18-00-01Z.pb'

In [ ]:
filenames = glob("./data/1-koda-gtfs-rt-extracted/2021-04-01T00/sl/VehiclePositions/2021/04/01/18/*.pb")

details = []

for file in tqdm(filenames): 
    response = open(filename, 'rb')
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.read())
    
    details += [
        (e.id, feed.header.timestamp, e.vehicle.trip.trip_id, e.vehicle.vehicle.id, e.vehicle.timestamp, 
         Point(e.vehicle.position.longitude, e.vehicle.position.latitude))
        for e in feed.entity
    ]
    break;

In [ ]:
gdf = gpd.GeoDataFrame(details, columns=['eid', 'timestamp', 'tid', 'vid', 'ts', 'geometry'])
gdf.set_crs(epsg=4326, inplace=True)
gdf

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
gdf.plot(ax=ax)
cx.add_basemap(ax, crs=gdf.crs)